In [1]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from pathlib import Path
import neptune # for logging progress and results
from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import set_environment_variables, neptune_output_figures
neptune_kwargs = {'project':"commander.pho/PhoDibaLongShort2023",
'api_token':"eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxOGIxODU2My1lZTNhLTQ2ZWMtOTkzNS02ZTRmNzM5YmNjNjIifQ=="}

# required to enable non-blocking interaction:
%gui qt5

from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd
from attrs import define, field, fields, Factory

# Pho's Formatting Preferences
# from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
# set_pho_preferences_concise()

## Pho's Custom Libraries:
from pyphocorehelpers.general_helpers import CodeConversion
from pyphocorehelpers.function_helpers import function_attributes
from pyphocorehelpers.print_helpers import print_keys_if_possible, print_value_overview_only, document_active_variables, objsize, print_object_memory_usage, debug_dump_object_member_shapes, TypePrintMode
from pyphocorehelpers.print_helpers import get_now_day_str, get_now_time_str, get_now_time_precise_str
from pyphocorehelpers.programming_helpers import IPythonHelpers
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager

# NeuroPy (Diba Lab Python Repo) Loading
# from neuropy import core
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.core.epoch import NamedTimerange, Epoch
from neuropy.core.ratemap import Ratemap
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass

## For computation parameters:
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.utils.dynamic_container import DynamicContainer
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core.neurons import NeuronType

from pyphocorehelpers.print_helpers import generate_html_string

# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot, create_daily_programmatic_display_function_testing_folder_if_needed, session_context_to_relative_path
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_load_session, batch_extended_computations, SessionBatchProgress, batch_programmatic_figures, batch_extended_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap
from pyphoplacecellanalysis.Pho2D.PyQtPlots.Extensions.pyqtgraph_helpers import build_pyqtgraph_epoch_indicator_regions # Add session indicators to pyqtgraph plot
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _helper_add_long_short_session_indicator_regions
import pyphoplacecellanalysis.External.pyqtgraph as pg
# 'time_bin_indices': valid_time_bin_indicies, 'posterior_to_pf_mean_surprise': one_left_out_posterior_to_pf_surprises_mean, 'posterior_to_scrambled_pf_mean_surprise': one_left_out_posterior_to_scrambled_pf_surprises_mean}
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_long_short_expected_vs_observed_firing_rates
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_long_short_surprise_difference_plot

from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import _update_pipeline_missing_preprocessing_parameters
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons

from scripts.run_BatchAnalysis import post_compute_validate, _on_complete_success_execution_session

session_batch_status = {}
session_batch_errors = {}
enable_saving_to_disk = False

global_data_root_parent_path = find_first_extant_path([Path(r'W:\Data'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

Automatic pdb calling has been turned OFF
build_module_logger(module_name="Spike3D.pipeline"):
	 Module logger com.PhoHale.Spike3D.pipeline has file logging enabled and will log to EXTERNAL\TESTING\Logging\debug_com.PhoHale.Spike3D.pipeline.log


# Load Pipeline

In [2]:
# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

active_data_mode_name = 'kdiba'

## Data must be pre-processed using the MATLAB script located here: 
#     neuropy/data_session_pre_processing_scripts/KDIBA/IIDataMat_Export_ToPython_2022_08_01.m
# From pre-computed .mat files:

local_session_root_parent_context = IdentifyingContext(format_name=active_data_mode_name) # , animal_name='', configuration_name='one', session_name=self.session_name
local_session_root_parent_path = global_data_root_parent_path.joinpath('KDIBA')

good_contexts_list = [
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54')
]

from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass




In [3]:
%pdb off
# %%viztracer
# [*] - indicates bad or session with a problem
# 0, 1, 2, 3, 4, 5, 6, 7, [8], [9], 10, 11, [12], 13, 14, [15], [16], 17, 
# curr_context: IdentifyingContext = good_contexts_list[1] # select the session from all of the good sessions here.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15') # DONE. Very good. Many good Pfs, many good replays.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43') # BAD
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31') # DONE, Good Pfs but no good replays
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6') # BAD
curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3') # DONE, Good Pfs, no good epochs.
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40') # BAD
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3') # BAD, Good Pfs strangely despite horrible map, no good epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59') # BAD, No Epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54') # DONE


local_session_parent_path: Path = local_session_root_parent_path.joinpath(curr_context.animal, curr_context.exper_name) # 'gor01', 'one' - probably not needed anymore
basedir: Path = local_session_parent_path.joinpath(curr_context.session_name).resolve()
print(f'basedir: {str(basedir)}')

# Read if possible:
# saving_mode = PipelineSavingScheme.SKIP_SAVING
# force_reload = False

# Force write:
# saving_mode = PipelineSavingScheme.TEMP_THEN_OVERWRITE
saving_mode = PipelineSavingScheme.OVERWRITE_IN_PLACE
force_reload = True


## TODO: if loading is not possible, we need to change the `saving_mode` so that the new results are properly saved.


    # ==================================================================================================================== #
    # Load Pipeline                                                                                                        #
    # ==================================================================================================================== #
# with VizTracer(output_file=f"viztracer_{get_now_time_str()}-full_session_LOO_decoding_analysis.json", min_duration=200, tracer_entries=3000000, ignore_frozen=True) as tracer:
# epoch_name_includelist = ['maze']
epoch_name_includelist = None
active_computation_functions_name_includelist=[#'_perform_estimated_epochs_computation',  # AL:WAYS OFF
                                            '_perform_baseline_placefield_computation',
                                        # '_perform_time_dependent_placefield_computation', # AL:WAYS OFF
                                        '_perform_extended_statistics_computation',
                                        '_perform_position_decoding_computation', 
                                        '_perform_firing_rate_trends_computation',
                                        '_perform_pf_find_ratemap_peaks_computation',
                                        # '_perform_time_dependent_pf_sequential_surprise_computation' # AL:WAYS OFF
                                        '_perform_two_step_position_decoding_computation',
                                        # '_perform_recursive_latent_placefield_decoding' # AL:WAYS OFF
                                    ]
# active_computation_functions_name_includelist=['_perform_baseline_placefield_computation']

curr_active_pipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, basedir, epoch_name_includelist=epoch_name_includelist,
                                        computation_functions_name_includelist=active_computation_functions_name_includelist,
                                        saving_mode=saving_mode, force_reload=force_reload,
                                        skip_extended_batch_computations=True, debug_print=False, fail_on_exception=True) # , active_pickle_filename = 'loadedSessPickle_withParameters.pkl'


if not force_reload: # not just force_reload, needs to recompute whenever the computation fails.
    try:
        curr_active_pipeline.load_pickled_global_computation_results()
    except Exception as e:
        print(f'cannot load global results: {e}')

# 6m 1.1s
# 12m 15.6s


Automatic pdb calling has been turned OFF
basedir: W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19
Skipping loading from pickled file because force_reload == True.
Loading matlab import file results : W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\2006-6-07_16-40-19.epochs_info.mat... done.
Loading matlab import file results : W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\2006-6-07_16-40-19.position_info.mat... done.
Loading matlab import file results : W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\2006-6-07_16-40-19.spikes.mat... 

C:\Users\pho\repos\Spike3DWorkEnv\NeuroPy\neuropy\core\session\Formats\SessionSpecifications.py:140: UserWarning: WARNING: Optional File: W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\2006-6-07_16-40-19.dat does not exist. Continuing without it.
  warnings.warn(f'WARNING: Optional File: {an_optional_filepath} does not exist. Continuing without it.')


done.
Failure loading .position.npy. Must recompute.



c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\sklearn\manifold\_isomap.py:352: UserWarning: The number of connected components of the neighbors graph is 2 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\scipy\sparse\_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


Saving updated position results results : W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\2006-6-07_16-40-19.position.npy... 2006-6-07_16-40-19.position.npy saved
done.
	 force_recompute is True! Forcing recomputation of .interpolated_spike_positions.npy

Computing interpolate_spike_positions columns results : spikes_df... done.
	 Saving updated interpolated spike position results results : W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\2006-6-07_16-40-19.interpolated_spike_positions.npy... 2006-6-07_16-40-19.interpolated_spike_positions.npy saved
done.
Loading matlab import file results : W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\2006-6-07_16-40-19.laps_info.mat... done.
setting laps object.
session.laps loaded successfully!
Loading matlab import file results : W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\2006-6-07_16-40-19.replay_info.mat... done.
session.replays loaded successfully!
Loading success: W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\ripple_df.pkl.
Failure loading .mua.npy. Must recom

...

Applying session filter named "maze2"...
Constraining to epoch with times (start: 1236.2662453636294, end: 2587.801681999932)
computing neurons mua for session...



C:\Users\pho\repos\Spike3DWorkEnv\NeuroPy\neuropy\core\laps.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_df[['lap_id']] = laps_df[['lap_id']].astype('int')
C:\Users\pho\repos\Spike3DWorkEnv\NeuroPy\neuropy\core\laps.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_df[['start_spike_index', 'end_spike_index']] = laps_df[['start_spike_index', 'end_spike_index']].astype('int')
C:\Users\pho\repos\Spike3DWorkEnv\NeuroPy\neuropy\core\laps.py:70: SettingWithCopyWarning: 
A value is trying to b

Applying session filter named "maze"...
Constraining to epoch with times (start: 0.0, end: 2587.801681999932)
computing neurons mua for session...

using provided computation_functions_name_includelist: ['_perform_baseline_placefield_computation', '_perform_extended_statistics_computation', '_perform_position_decoding_computation', '_perform_firing_rate_trends_computation', '_perform_pf_find_ratemap_peaks_computation', '_perform_two_step_position_decoding_computation']
due to includelist, including only 6 out of 16 registered computation functions.
Recomputing active_epoch_placefields... using self.config.grid_bin_bounds_1D: (22.397021260868584, 245.6584673739576)
	 done.
Recomputing active_epoch_placefields2D... using self.config.grid_bin_bounds: ((22.397021260868584, 245.6584673739576), (133.66465594522782, 155.97244934208123))
	 done.
	 spikes_df[time_variable_name]: (15034,) should be less than time_window_edges: (35780,)!
	 spikes_df[time_variable_name]: (15034,) should be less th

C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\Analysis\Decoder\reconstruction.py:363: RuntimeWarning: divide by zero encountered in divide
  return C * np.exp(numerator/denominator)


two_step_decoder_result['most_likely_position_flat_max_likelihood_values'].shape = (36136,)
updating computation_results...
done.
due to includelist, including only 6 out of 16 registered computation functions.
Recomputing active_epoch_placefields... using self.config.grid_bin_bounds_1D: (22.397021260868584, 245.6584673739576)
	 done.
Recomputing active_epoch_placefields2D... using self.config.grid_bin_bounds: ((22.397021260868584, 245.6584673739576), (133.66465594522782, 155.97244934208123))
	 done.
	 spikes_df[time_variable_name]: (12793,) should be less than time_window_edges: (38260,)!
	 spikes_df[time_variable_name]: (12793,) should be less than time_window_edges: (38260,)!
two_step_decoder_result['most_likely_position_flat_max_likelihood_values'].shape = (38641,)
updating computation_results...
done.
due to includelist, including only 6 out of 16 registered computation functions.
Recomputing active_epoch_placefields... using self.config.grid_bin_bounds_1D: (22.397021260868584, 24

In [4]:
%matplotlib qt
from neuropy.utils.matplotlib_helpers import interactive_select_grid_bin_bounds_2D
# fig, ax, rect_selector, set_extents = interactive_select_grid_bin_bounds_2D(curr_active_pipeline, epoch_name='maze', should_block_for_input=True)

grid_bin_bounds = interactive_select_grid_bin_bounds_2D(curr_active_pipeline, epoch_name='maze', should_block_for_input=True, should_apply_updates_to_pipeline=False)
print(f'grid_bin_bounds: {grid_bin_bounds}')
print(f"Add this to `specific_session_override_dict`:\n\n{curr_active_pipeline.get_session_context().get_initialization_code_string()}:dict(grid_bin_bounds=({(grid_bin_bounds[0], grid_bin_bounds[1]), (grid_bin_bounds[2], grid_bin_bounds[3])})),\n")

blocking and waiting for user input. Press [enter] to confirm selection change or [esc] to revert with no change.
user confirmed extents: (22.397021260868584, 245.3970212608686, 133.66465594522782, 155.97244934208123)
Add this to `specific_session_override_dict`:

IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'):dict(grid_bin_bounds=(((22.397021260868584, 245.3970212608686), (133.66465594522782, 155.97244934208123)))),

grid_bin_bounds: (22.397021260868584, 245.3970212608686, 133.66465594522782, 155.97244934208123)
Add this to `specific_session_override_dict`:

IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'):dict(grid_bin_bounds=(((22.397021260868584, 245.3970212608686), (133.66465594522782, 155.97244934208123)))),



In [5]:
## Post Compute Validate 2023-05-16:
from scripts.run_BatchAnalysis import post_compute_validate

post_compute_validate(curr_active_pipeline)

were pipeline preprocessing parameters missing and updated?: False


In [6]:
curr_active_pipeline.reload_default_computation_functions()
extended_computations_include_includelist=['long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis', 'long_short_decoding_analyses', 'long_short_post_decoding'] # do only specifiedl , 'long_short_rate_remapping'
force_recompute_global = force_reload
# force_recompute_global = True
# extended_computations_include_includelist=['long_short_post_decoding'] # do only specifiedl
newly_computed_values = batch_extended_computations(curr_active_pipeline, include_includelist=extended_computations_include_includelist, include_global_functions=True, fail_on_exception=True, progress_print=True, force_recompute=force_recompute_global, debug_print=False)
if len(newly_computed_values) > 0:
	print(f'newly_computed_values: {newly_computed_values}. Saving global results...')
	try:
		# curr_active_pipeline.global_computation_results.persist_time = datetime.now()
		# Try to write out the global computation function results:
		curr_active_pipeline.save_global_computation_results()
	except Exception as e:
		print(f'\n\n!!WARNING!!: saving the global results threw the exception: {e}')
		print(f'\tthe global results are currently unsaved! proceed with caution and save as soon as you can!\n\n\n')
else:
	print(f'no changes in global results.')
# 10m 29.5s

included includelist is specified: ['long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis', 'long_short_decoding_analyses', 'long_short_post_decoding'], so only performing these extended computations.
Running batch_extended_computations(...) with global_epoch_name: "maze"
jonathan_firing_rate_analysis missing.
	 Recomputing jonathan_firing_rate_analysis...
	 done.
long_short_fr_indicies_analyses missing.
	 Recomputing long_short_fr_indicies_analyses...
	 done.
long_short_decoding_analyses missing.
	 Recomputing long_short_decoding_analyses...
sane_midpoint_x: 137.60573860249627, hardcoded_track_midpoint_x: None, track_min_max_x: (18.67140495721134, 256.5400722477812)
hardcoded_track_midpoint_x is None, falling back to sane_midpoint_x... 137.60573860249627
desc_crossings_x: (21,), asc_crossings_x: (21,)
setting new computation epochs because laps changed.
using self.config.grid_bin_bounds_1D: (22.397021260868584, 245.6584673739576)
using self.config.grid_bin_bounds: ((22.397

In [7]:
curr_active_pipeline.reload_default_display_functions()

neptuner = batch_perform_all_plots(curr_active_pipeline, enable_neptune=True)


https://app.neptune.ai/commander.pho/PhoDibaLongShortUpdated/
https://app.neptune.ai/commander.pho/PhoDibaLongShortUpdated/e/LS2023-151
shared_aclus: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]
long_only_aclus: [12, 21, 48]
short_only_aclus: [18, 19, 65]
shared_aclus: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]
long_only_aclus: [12, 21, 48]
short_only_aclus: [18, 19, 65]
include_includelist: ['maze1', 'maze2', 'maze']
long_epoch_name: maze1, short_epoch_name: maze2, global_epoch_name: maze
	 saved C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\Screenshots\ProgrammaticDisplayFunctionTesting\2023-06-20\kdiba_gor01_two_2006-6-07_

on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =12
	selection_indicies: [12]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =13
	selection_indicies: [12 13]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =15
	selection_indicies: [12 13 15]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =17
	selection_indicies: [12 13 15 17]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =20
	selection_indicies: [12 13 15 17 20]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =24
	selection_indicies: [12 13 15 17 20 24]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =5
	selection_indicies: [5]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =13
	selection_indicies: [ 5 13]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =15
	selection_indicies: [ 5 13 15]
on_selected_epochs_changed(...)
current

### Pipeline Loading/Saving [Optional]

In [ ]:
curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE) # AttributeError: 'PfND_TimeDependent' object has no attribute '_included_thresh_neurons_indx'
# TypeError: cannot pickle 'MplMultiTab' object
# PicklingError: Can't pickle .set_closure_cell at 0x000002BF248F50D0>: it's not found as attr._compat.make_set_closure_cell..set_closure_cell
# TypeError: cannot pickle 'PyQt5.QtCore.pyqtSignal' object

In [ ]:
curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.OVERWRITE_IN_PLACE)

In [ ]:
curr_active_pipeline.save_global_computation_results() # PicklingError: Can't pickle .set_closure_cell at 0x000002BF248F50D0>: it's not found as attr._compat.make_set_closure_cell..set_closure_cell

In [ ]:
curr_active_pipeline.load_pickled_global_computation_results()

In [ ]:
curr_active_pipeline.clear_display_outputs()
curr_active_pipeline.clear_registered_output_files()

In [ ]:
curr_active_pipeline.save_global_computation_results()

# End Run

In [8]:
## long_short_decoding_analyses:
curr_long_short_decoding_analyses = curr_active_pipeline.global_computation_results.computed_data['long_short_leave_one_out_decoding_analysis']
## Extract variables from results object:
long_one_step_decoder_1D, short_one_step_decoder_1D, long_replays, short_replays, global_replays, long_shared_aclus_only_decoder, short_shared_aclus_only_decoder, shared_aclus, long_short_pf_neurons_diff, n_neurons, long_results_obj, short_results_obj, is_global = curr_long_short_decoding_analyses.long_decoder, curr_long_short_decoding_analyses.short_decoder, curr_long_short_decoding_analyses.long_replays, curr_long_short_decoding_analyses.short_replays, curr_long_short_decoding_analyses.global_replays, curr_long_short_decoding_analyses.long_shared_aclus_only_decoder, curr_long_short_decoding_analyses.short_shared_aclus_only_decoder, curr_long_short_decoding_analyses.shared_aclus, curr_long_short_decoding_analyses.long_short_pf_neurons_diff, curr_long_short_decoding_analyses.n_neurons, curr_long_short_decoding_analyses.long_results_obj, curr_long_short_decoding_analyses.short_results_obj, curr_long_short_decoding_analyses.is_global

# (long_one_step_decoder_1D, short_one_step_decoder_1D), (long_one_step_decoder_2D, short_one_step_decoder_2D) = compute_short_long_constrained_decoders(curr_active_pipeline, recalculate_anyway=True)
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
long_epoch_context, short_epoch_context, global_epoch_context = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_epoch_name, short_epoch_name, global_epoch_name)]
long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_computation_config, short_computation_config, global_computation_config = [curr_active_pipeline.computation_results[an_epoch_name]['computation_config'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_pf1D, short_pf1D, global_pf1D = long_results.pf1D, short_results.pf1D, global_results.pf1D
long_pf2D, short_pf2D, global_pf2D = long_results.pf2D, short_results.pf2D, global_results.pf2D
decoding_time_bin_size = long_one_step_decoder_1D.time_bin_size # 1.0/30.0 # 0.03333333333333333

## Get global 'long_short_post_decoding' results:
curr_long_short_post_decoding = curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
expected_v_observed_result, curr_long_short_rr = curr_long_short_post_decoding.expected_v_observed_result, curr_long_short_post_decoding.rate_remapping
rate_remapping_df, high_remapping_cells_only = curr_long_short_rr.rr_df, curr_long_short_rr.high_only_rr_df
Flat_epoch_time_bins_mean, Flat_decoder_time_bin_centers, num_neurons, num_timebins_in_epoch, num_total_flat_timebins, is_short_track_epoch, is_long_track_epoch, short_short_diff, long_long_diff = expected_v_observed_result['Flat_epoch_time_bins_mean'], expected_v_observed_result['Flat_decoder_time_bin_centers'], expected_v_observed_result['num_neurons'], expected_v_observed_result['num_timebins_in_epoch'], expected_v_observed_result['num_total_flat_timebins'], expected_v_observed_result['is_short_track_epoch'], expected_v_observed_result['is_long_track_epoch'], expected_v_observed_result['short_short_diff'], expected_v_observed_result['long_long_diff']

jonathan_firing_rate_analysis_result = JonathanFiringRateAnalysisResult(**curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.to_dict())

# 2023-05-19 - Testing S-only emergence, L-only replays in S, peak position remappings 

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult

jonathan_firing_rate_analysis_result = JonathanFiringRateAnalysisResult(**curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.to_dict())
neuron_replay_stats_df = jonathan_firing_rate_analysis_result.neuron_replay_stats_df.copy()
neuron_replay_stats_df

In [ ]:
neuron_replay_stats_df.sort_values(by=['long_pf_peak_x'], inplace=False, ascending=True)

In [ ]:
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership, SetPartition
@define(slots=False)
class TrackPartitioningResult:
	is_S_pf_only: np.ndarray
	is_S_only: np.ndarray
	S_only_aclus: np.ndarray
	S_only_df: pd.DataFrame

# Four distinct subgroups are formed:  pf on neither, pf on both, pf on only long, pf on only short
# L_only_aclus, S_only_aclus

In [ ]:
## 2023-05-19 - Get S-only pfs
is_S_pf_only = np.logical_and(np.logical_not(neuron_replay_stats_df['has_long_pf']), neuron_replay_stats_df['has_short_pf'])
is_S_only = neuron_replay_stats_df.track_membership == SplitPartitionMembership.RIGHT_ONLY
assert (is_S_pf_only == is_S_only).all()
S_only_aclus = neuron_replay_stats_df.index[is_S_only].to_numpy()
neuron_replay_stats_df[is_S_pf_only]

In [ ]:
## Show L-only pfs stop replaying on S
is_L_pf_only = np.logical_and(np.logical_not(neuron_replay_stats_df['has_short_pf']), neuron_replay_stats_df['has_long_pf'])
is_L_only = neuron_replay_stats_df.track_membership == SplitPartitionMembership.LEFT_ONLY
assert (is_L_pf_only == is_L_only).all()
L_only_aclus = neuron_replay_stats_df.index[is_L_only].to_numpy()
neuron_replay_stats_df[is_L_only]


## For ('kdiba', 'gor01', 'one', '2006-6-09_1-22-43') - Have L-only cells [24, 98] that have ['short_num_replays'] = [8, 7]. We were hoping that there would be few to no replays on the S-track that involved L-only cells.

In [ ]:
## 2023-05-23 - Get Common (SHARED) placefields
## Goal 1: From the cells with the placefields on both tracks, compute the degree to which they remap in position and sort them according to their distance.
is_BOTH_pf_only = np.logical_and(neuron_replay_stats_df['has_short_pf'], neuron_replay_stats_df['has_long_pf']) # (63,)
BOTH_pf_only_aclus = neuron_replay_stats_df.index[is_BOTH_pf_only].to_numpy()

## NOTE: is_BOTH_pf_only is a much more stringent requirement (and a strict subset) than `is_BOTH_only`
is_BOTH_only = neuron_replay_stats_df.track_membership == SplitPartitionMembership.SHARED # (99,)
BOTH_only_aclus = neuron_replay_stats_df.index[is_BOTH_only].to_numpy()
assert BOTH_only_aclus.shape[0] >= BOTH_pf_only_aclus.shape[0]

BOTH_pf_only_df = neuron_replay_stats_df[is_BOTH_pf_only].copy()
BOTH_pf_only_df['long_short_pf_peak_x_displacement'] = BOTH_pf_only_df['long_pf_peak_x'].values - BOTH_pf_only_df['short_pf_peak_x'].values
BOTH_pf_only_df['long_short_pf_peak_x_distance'] = BOTH_pf_only_df['long_short_pf_peak_x_displacement'].abs()
BOTH_pf_only_df.sort_values(by=['long_short_pf_peak_x_distance'], inplace=True, ascending=False)
BOTH_pf_only_df

#TODO 2023-05-23 - Can do more detailed peaks analysis with: long_results.RatemapPeaksAnalysis and short_results.RatemapPeaksAnalysis



In [ ]:
# 2023-06-19 - Look at the Long and Short only aclus and find replays in which they replay
# L_only_aclus, S_only_aclus

# long_replays

exclusive_aclus = np.hstack((L_only_aclus, S_only_aclus)) # [  2,   8,  19,  28,  36,  54,  63,  68,  73,  74, 109,   4,  13,  21,  23,  52,  58,  67,  70,  81]
exclusive_aclus
# find the cells that fire in each replay. I think this is already computed/used somewhere


In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import _prepare_spikes_df_from_filter_epochs
# approach copied from `pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations._epoch_unit_avg_firing_rates`
from pyphocorehelpers.indexing_helpers import partition

# included_neuron_ids = deepcopy(exclusive_aclus)
included_neuron_ids = None
spikes_df = deepcopy(curr_active_pipeline.sess.spikes_df)
filter_epochs = deepcopy(curr_active_pipeline.sess.replay)

if included_neuron_ids is None:
	included_neuron_ids = spikes_df.spikes.neuron_ids

if isinstance(filter_epochs, pd.DataFrame):
	filter_epochs_df = filter_epochs
else:
	filter_epochs_df = filter_epochs.to_dataframe()
	
filter_epoch_spikes_df = deepcopy(spikes_df)
filter_epoch_spikes_df = _prepare_spikes_df_from_filter_epochs(filter_epoch_spikes_df, filter_epochs=filter_epochs_df, included_neuron_ids=included_neuron_ids, epoch_id_key_name='replay_epoch_id', debug_print=False) # replay_epoch_id
filter_epoch_spikes_df


In [ ]:
filter_epochs # 364



In [ ]:
unique_replay_epoch_id_values, epoch_split_spikes_df_list = partition(filter_epoch_spikes_df, 'replay_epoch_id')
# filter_epoch_spikes_df.groupby('replay_epoch_id')
unique_replay_epoch_id_values # (198,)

In [ ]:
epoch_spikes_unique_aclus_list = []
epoch_contains_any_exclusive_aclus = []

for an_epoch_id, epoch_specific_spikes_df in zip(unique_replay_epoch_id_values, epoch_split_spikes_df_list):
    # Loop through the epochs
	# find epochs containing at least one in `exclusive_aclus`:
	
	epoch_spikes_unique_aclus = epoch_specific_spikes_df.aclu.unique()
	epoch_spikes_unique_aclus_list.append(epoch_spikes_unique_aclus)
	# epoch_spikes_unique_aclus # finds lists of all the active aclus in the given epoch
	if len(epoch_spikes_unique_aclus) > 0:	
		epoch_contains_any_exclusive_aclus.append(np.isin(epoch_spikes_unique_aclus, exclusive_aclus).any())
	else:
		epoch_contains_any_exclusive_aclus.append(False)
	# epoch_contains_any_exclusive_aclus # returns True if the epoch contains one ore more spikes from the search list `exclusive_aclus`

epoch_contains_any_exclusive_aclus

In [ ]:

# find epochs containing at least one in `exclusive_aclus`:
epoch_spikes_unique_aclus = [epoch_split_spikes_df.aclu.unique() for epoch_split_spikes_df in epoch_split_spikes_df_list]
# epoch_spikes_unique_aclus # finds lists of all the active aclus in the given epoch
epoch_contains_any_exclusive_aclus = [np.isin(epoch_aclus, exclusive_aclus).any() for epoch_aclus in epoch_spikes_unique_aclus]
# epoch_contains_any_exclusive_aclus # returns True if the epoch contains one ore more spikes from the search list `exclusive_aclus`
filter_epochs['contains_one_exclusive_aclu'] = epoch_contains_any_exclusive_aclus
# epoch_contains_any_exclusive_aclus

In [ ]:
replay_epoch_IDXs_with_exclusive_aclu = filter_epoch_spikes_df.replay_epoch_id.unique() # finds the epochs with at least one aclu from the `exclusive_aclus` list in them


In [ ]:
# first slice only to the neurons of interest:
# filter_epoch_spikes_df


In [ ]:
""" 24 cells, 133 Epochs, 789 Total Timebins
# Other
	
	- is_non_firing_time_bin: numpy.ndarray - (24, 789)
	- all_epochs_num_epoch_time_bins: numpy.ndarray - (133,)
	
    ## Indexing Helpers:
		- all_epochs_reverse_flat_epoch_indicies_array: numpy.ndarray - (789,)
		- split_by_epoch_reverse_flattened_time_bin_indicies: list - (133,)
    
	- original_1D_decoder: pyphoplacecellanalysis.Analysis.Decoder.reconstruction.BasePositionDecoder
		- pf: neuropy.analyses.placefields.PfND
		- neuron_IDXs: numpy.ndarray - (24,)
		- neuron_IDs: numpy.ndarray - (24,)
		- F: numpy.ndarray - (119, 24)
		- P_x: numpy.ndarray - (119, 1)
        
	- flat_all_epochs_decoded_epoch_time_bins: numpy.ndarray - (24, 789)

# Measured
	- flat_all_epochs_measured_cell_spike_counts: numpy.ndarray - (24, 789)
	- flat_all_epochs_measured_cell_firing_rates: numpy.ndarray - (24, 789)


# Expected
	- flat_all_epochs_computed_expected_cell_firing_rates: numpy.ndarray - (24, 789)
	- flat_all_epochs_difference_from_expected_cell_spike_counts: numpy.ndarray - (24, 789)
	- flat_all_epochs_difference_from_expected_cell_firing_rates: numpy.ndarray - (24, 789)
    
    
    
## Epoch-based
	- all_epochs_decoded_epoch_time_bins_mean: numpy.ndarray - (133, 24)

    
    
    
- flat_all_epochs_measured_cell_firing_rates: numpy.ndarray - .shape: (24, 789) 
- flat_all_epochs_computed_expected_cell_firing_rates: numpy.ndarray - .shape: (24, 789)


"""

# 2023-04-13 - Find Good looking epochs:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib qt
import matplotlib.patheffects as path_effects

from pyphoplacecellanalysis.Pho2D.stacked_epoch_slices import DecodedEpochSlicesPaginatedFigureController

curr_results_obj = long_results_obj
active_identifying_session_ctx = curr_active_pipeline.sess.get_context()
_out_pagination_controller = DecodedEpochSlicesPaginatedFigureController.init_from_decoder_data(curr_results_obj.active_filter_epochs, curr_results_obj.all_included_filter_epochs_decoder_result, 
	xbin=curr_results_obj.original_1D_decoder.xbin, global_pos_df=global_session.position.df, a_name='TestDecodedEpochSlicesPaginationController', active_context=active_identifying_session_ctx,  max_subplots_per_page=200) # 10
# _out_pagination_controller

In [10]:
## Getting selections out of DecoedEpochSlicesPaginationController
curr_active_pipeline.display_output_history_list
# [IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze1')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'long_only', '[2, 8, 19, 28, 36, 54, 63, 68, 73, 74, 109]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'short_only', '[4, 13, 21, 23, 52, 58, 67, 70, 81]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '1of9', '[3, 5, 6, 7, 9, 10, 11, 12, 14, 15]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '2of9', '[16, 17, 18, 20, 22, 24, 25, 26, 27, 29]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '3of9', '[30, 31, 32, 33, 34, 35, 37, 38, 39, 40]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '4of9', '[41, 42, 43, 44, 45, 46, 47, 48, 49, 50]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '5of9', '[51, 53, 55, 56, 57, 59, 60, 61, 62, 64]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '6of9', '[65, 66, 69, 71, 72, 75, 76, 77, 78, 79]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '7of9', '[80, 82, 83, 84, 85, 86, 87, 88, 89, 90]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '8of9', '[91, 92, 93, 94, 95, 96, 97, 98, 99, 100]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '9of9', '[101, 102, 103, 104, 105, 106, 107, 108]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze1', '_display_1d_placefields')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze2', '_display_1d_placefields')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze', '_display_1d_placefields')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze1', '_display_1d_placefield_validations')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze2', '_display_1d_placefield_validations')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze', '_display_1d_placefield_validations')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze1', '_display_2d_placefield_result_plot_ratemaps_2D')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze2', '_display_2d_placefield_result_plot_ratemaps_2D')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze', '_display_2d_placefield_result_plot_ratemaps_2D')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze1', '_display_2d_placefield_occupancy')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze2', '_display_2d_placefield_occupancy')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze', '_display_2d_placefield_occupancy')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', '_display_short_long_firing_rate_index_comparison')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', '_display_long_short_expected_v_observed_firing_rate')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', '_display_grid_bin_bounds_validation')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', '_display_running_and_replay_speeds_over_time')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', '_display_long_short_laps')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', '_display_long_and_short_firing_rate_replays_v_laps')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', '_display_short_long_pf1D_comparison')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', '_display_long_and_short_stacked_epoch_slices')>]

# Ideally could search like: '_display_long_and_short_stacked_epoch_slices'
active_result_backup = curr_active_pipeline.last_added_display_output

In [11]:
list(active_result_backup.keys()) # ['name', 'context', 'figures', 'axes', 'plot_data']

active_result_backup['context'] # (IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'DecodedEpochSlices', 'replays', 'long_results_obj')>,
								#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'DecodedEpochSlices', 'replays', 'short_results_obj')>)

pagination_controllers = active_result_backup.plot_data['controllers']

In [12]:
long_controller = pagination_controllers[0]
short_controller = pagination_controllers[1]

In [32]:
long_controller.selected_indicies # array([ 13,  14,  15,  25,  27,  28,  31,  37,  42,  45,  48,  57,  61,  62,  63,  76,  79,  82,  89,  90, 111, 112, 113, 115])


# np.array([  5,  13,  15,  17,  20,  21,  24,  31,  33,  43,  44,  49,  63, 64,  66,  68,  70,  71,  74,  76,  77,  78,  84,  90,  94,  95, 104, 105, 122, 123])

array([  5,  13,  15,  17,  20,  21,  24,  31,  33,  43,  44,  49,  63,
        64,  66,  68,  70,  71,  74,  76,  77,  78,  84,  90,  94,  95,
       104, 105, 122, 123])

In [31]:
short_controller.selected_indicies # array([  9,  11,  13,  14,  15,  20,  22,  25,  37,  40,  45,  48,  61, 62,  76,  79,  84,  89,  90,  93,  94, 111, 112, 113, 115, 121])

# np.array([ 12,  13,  15,  17,  20,  24,  30,  31,  32,  33,  41,  43,  49, 54,  55,  68,  70,  71,  73,  76,  77,  78,  84,  89,  94, 100, 104, 105, 111, 114, 115, 117, 118, 122, 123, 131])

array([ 12,  13,  15,  17,  20,  24,  30,  31,  32,  33,  41,  43,  49,
        54,  55,  68,  70,  71,  73,  76,  77,  78,  84,  89,  94, 100,
       104, 105, 111, 114, 115, 117, 118, 122, 123, 131])

In [33]:
assert long_controller.is_selected.shape[0] == long_results_obj.active_filter_epochs.n_epochs
assert short_controller.is_selected.shape[0] == short_results_obj.active_filter_epochs.n_epochs
# for some reason the active_filter_epochs on both the long and the short objects contain properties about the other one as well, so keep with tradition and add them to both as well
long_results_obj.active_filter_epochs._df['long_is_user_included'] = long_controller.is_selected
long_results_obj.active_filter_epochs._df['short_is_user_included'] = short_controller.is_selected
short_results_obj.active_filter_epochs._df['long_is_user_included'] = long_controller.is_selected
short_results_obj.active_filter_epochs._df['short_is_user_included'] = short_controller.is_selected

In [ ]:
long_controller.is_selected, long_controller.selected_indicies, long_controller.plots_data.epoch_slices, long_controller.params.flat_all_data_indicies

In [21]:
print(list(long_controller.params.keys())) # ['name', 'window_title', 'num_slices', '_debug_test_max_num_slices', 'active_num_slices', 'global_epoch_start_t', 'global_epoch_end_t', 'single_plot_fixed_height', 'all_plots_height', 'epoch_labels', 'variable_name', 'xbin', 'enable_flat_line_drawing', 'debug_print', 'active_identifying_figure_ctx', 'flat_all_data_indicies', 'is_selected', 'callback_id', 'on_render_page_callbacks']



['name', 'window_title', 'num_slices', '_debug_test_max_num_slices', 'active_num_slices', 'global_epoch_start_t', 'global_epoch_end_t', 'single_plot_fixed_height', 'all_plots_height', 'epoch_labels', 'variable_name', 'xbin', 'enable_flat_line_drawing', 'debug_print', 'active_identifying_figure_ctx', 'flat_all_data_indicies', 'is_selected', 'callback_id', 'on_render_page_callbacks']


In [23]:
long_controller.params.active_identifying_figure_ctx.adding_context(user_annotation="selections")

TypeError: adding_context() missing 1 required positional argument: 'collision_prefix'

on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =111
	selection_indicies: [ 12  13  15  17  20  24  30  31  32  33  41  43  49  54  55  68  70  71
  73  76  77  78  84  89  94 100 104 105 111]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =68
	selection_indicies: [ 5 13 15 17 20 21 24 31 33 43 44 49 66 68 70 71]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =63
	selection_indicies: [ 5 13 15 17 20 21 24 31 33 43 44 49 63 66 68 70 71]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =64
	selection_indicies: [ 5 13 15 17 20 21 24 31 33 43 44 49 63 64 66 68 70 71]


In [16]:
long_controller.selected_epoch_times

array([[ 292.62367098,  292.80826736],
       [ 528.48329294,  528.68561475],
       [ 572.24842637,  572.44236163],
       [ 677.99306343,  678.25522777],
       [ 802.91157867,  803.11436129],
       [ 831.70962856,  831.90743454],
       [ 888.22678107,  888.46492239],
       [ 945.48035303,  945.68543965],
       [ 954.00517939,  954.2546871 ],
       [1033.05203044, 1033.22639707],
       [1061.35727035, 1061.5943979 ],
       [1214.61082388, 1214.834834  ],
       [1722.78022002, 1722.95406441],
       [1741.58530479, 1741.79413924],
       [1743.25171034, 1743.53445707]])

In [17]:
short_controller.selected_epoch_times

array([[ 489.42507549,  489.65575186],
       [ 528.48329294,  528.68561475],
       [ 572.24842637,  572.44236163],
       [ 677.99306343,  678.25522777],
       [ 802.91157867,  803.11436129],
       [ 888.22678107,  888.46492239],
       [ 931.21227202,  931.45262518],
       [ 945.48035303,  945.68543965],
       [ 950.18327543,  950.44046313],
       [ 954.00517939,  954.2546871 ],
       [1024.83173129, 1025.10320379],
       [1033.05203044, 1033.22639707],
       [1214.61082388, 1214.834834  ],
       [1326.54476977, 1326.76506278],
       [1329.84606216, 1330.0684134 ],
       [1732.09310613, 1732.29060491],
       [1741.58530479, 1741.79413924],
       [1743.25171034, 1743.53445707],
       [1751.03108555, 1751.22059714],
       [1769.90424581, 1770.10497019],
       [1776.56793264, 1776.78871716],
       [1871.00426701, 1871.21629634],
       [1917.09385877, 1917.2703758 ],
       [1994.07242543, 1994.2525367 ],
       [2038.53127755, 2038.70899266],
       [2064.36693792, 20

In [18]:
long_results_obj.active_filter_epochs.n_epochs # 122

133

# TODO 2023-06-19 14:31: - [ ] Come up with system for storing user-annotations:
 Need to store user-annotations and labeled selections for Epochs:

	date_annotated: datetime - the date the annotations were made
	annotation_author: str - the person who made the annotations
	annotation_content: Any - the content of the annotation



In [ ]:
# All epochs are shown for both decoders. So the result should be an epochs table with separate columns for the inclusion in each decoder.




# Other Programmatic Figures

In [ ]:
curr_active_pipeline.reload_default_display_functions()

In [ ]:
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons

## Builds proper sort indicies for '_display_short_long_pf1D_comparison'
long_ratemap = long_pf1D.ratemap
short_ratemap = short_pf1D.ratemap

curr_any_context_neurons = _find_any_context_neurons(*[k.neuron_ids for k in [long_ratemap, short_ratemap]])
curr_any_context_neurons

In [ ]:
from neuropy.plotting.ratemaps import plot_ratemap_1D
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_short_v_long_pf1D_comparison, LongShortTrackComparingDisplayFunctions

curr_active_pipeline.reload_default_display_functions()

%matplotlib qt
active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'

long_single_cell_pfmap_processing_fn = None
short_single_cell_pfmap_processing_fn = None
# sort_idx = None
# sort_idx = sort_ind
# sort_idx = sort_indicies
# sort_idx = new_all_aclus_sort_indicies
sort_idx = 'peak_long'
# sort_idx = 'bad'

# out = curr_active_pipeline.display('_display_short_long_pf1D_comparison', active_identifying_session_ctx, single_figure=False, debug_print=True, fignum='Short v Long pf1D Comparison',
#                                    long_kwargs={'included_unit_neuron_IDs': curr_any_context_neurons, 'included_unit_indicies': None, 'sortby': sort_idx, 'single_cell_pfmap_processing_fn': long_single_cell_pfmap_processing_fn},
#                                    short_kwargs={'included_unit_neuron_IDs': curr_any_context_neurons, 'included_unit_indicies': None,'sortby': sort_idx, 'single_cell_pfmap_processing_fn': short_single_cell_pfmap_processing_fn, 'curve_hatch_style': {'hatch':'///', 'edgecolor':'k'}},
#                                   )

out = curr_active_pipeline.display('_display_short_long_pf1D_comparison', active_identifying_session_ctx, single_figure=False, debug_print=False, fignum='Short v Long pf1D Comparison',
                                   long_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': long_single_cell_pfmap_processing_fn},
                                   short_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': short_single_cell_pfmap_processing_fn, 'curve_hatch_style': {'hatch':'///', 'edgecolor':'k'}},
                                  )
                                  

# ax = out.axes[0]

In [ ]:
curr_active_pipeline.display('_display_short_long_pf1D_comparison', single_figure=False)

# 2023-06-01 - Testing new metrics:

In [ ]:

curr_long_short_post_decoding = curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
## Extract variables from results object:
expected_v_observed_result, curr_long_short_rr = curr_long_short_post_decoding.expected_v_observed_result, curr_long_short_post_decoding.rate_remapping
rate_remapping_df, high_remapping_cells_only = curr_long_short_rr.rr_df, curr_long_short_rr.high_only_rr_df
Flat_epoch_time_bins_mean, Flat_decoder_time_bin_centers, num_neurons, num_timebins_in_epoch, num_total_flat_timebins, is_short_track_epoch, is_long_track_epoch, short_short_diff, long_long_diff = expected_v_observed_result['Flat_epoch_time_bins_mean'], expected_v_observed_result['Flat_decoder_time_bin_centers'], expected_v_observed_result['num_neurons'], expected_v_observed_result['num_timebins_in_epoch'], expected_v_observed_result['num_total_flat_timebins'], expected_v_observed_result['is_short_track_epoch'], expected_v_observed_result['is_long_track_epoch'], expected_v_observed_result['short_short_diff'], expected_v_observed_result['long_long_diff']

print(f'long_test: {np.mean(long_long_diff)}')
print(f'short_test: {np.mean(short_short_diff)}')

In [ ]:
long_session.replay

In [ ]:
# long_results_obj.original_1D_decoder
replay_result_df = deepcopy(long_results_obj.active_filter_epochs.to_dataframe())
replay_result_df

In [ ]:
short_results_obj.active_filter_epochs.to_dataframe()

In [ ]:
# Look at lap speed over time - doesn't look like there's a difference between the long and the short track speeds
global_session.position.compute_higher_order_derivatives()
running_pos_df = global_session.position.to_dataframe()
ax = running_pos_df.plot(x='t', y=['lin_pos','speed'], title='Running Speed over Time')
ax

In [ ]:
# Call the function
fig, ax = plot_lines(replay_result_df)

In [ ]:
import matplotlib
%matplotlib qt

curr_active_pipeline.reload_default_display_functions()
out_man = curr_active_pipeline.get_output_manager()
out_man.get_figure_output_path
curr_active_pipeline.write_figure_to_output_path



In [ ]:
_out = curr_active_pipeline.display('_display_grid_bin_bounds_validation')
# _display_running_and_replay_speeds_over_time


# 2023-06-13 11:01: - [ ] New display function testing

In [ ]:
import matplotlib as mpl
# from pyphoplacecellanalysis.General.Mixins.ExportHelpers import create_daily_programmatic_display_function_testing_folder_if_needed, session_context_to_relative_path, programmatic_display_to_PDF
# from pyphoplacecellanalysis.General.Mixins.ExportHelpers import build_pdf_metadata_from_display_context # newer version of build_pdf_export_metadata
from neuropy.core.neuron_identities import PlotStringBrevityModeEnum
from neuropy.plotting.ratemaps import BackgroundRenderingOptions
_bak_rcParams = mpl.rcParams.copy()
# mpl.rcParams['toolbar'] = 'None' # disable toolbars
# matplotlib.use('AGG') # non-interactive backend ## 2022-08-16 - Surprisingly this works to make the matplotlib figures render only to .png file, not appear on the screen!

%matplotlib qt

from pyphoplacecellanalysis.General.Mixins.ExportHelpers import programmatic_render_to_file

In [ ]:
curr_active_pipeline.reload_default_display_functions()


In [ ]:
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap, visualize_heatmap_pyqtgraph # used in `plot_kourosh_activity_style_figure`
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_raster_plot # used in `plot_kourosh_activity_style_figure`
import pyphoplacecellanalysis.External.pyqtgraph as pg # used in `plot_kourosh_activity_style_figure`

heatmap_pf1D_win, heatmap_pf1D_img = visualize_heatmap_pyqtgraph(long_pf1D.ratemap.normalized_tuning_curves, title=f"pf1D Visualization")
# Apply the colormap
# heatmap_pf1D_img.setLookupTable(lut)
heatmap_pf1D_win.show()

In [ ]:
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import _prepare_spikes_df_from_filter_epochs

long_ratemap = long_pf1D.ratemap
short_ratemap = short_pf1D.ratemap
curr_any_context_neurons = _find_any_context_neurons(*[k.neuron_ids for k in [long_ratemap, short_ratemap]])

filter_epochs = deepcopy(long_replays)

filter_epoch_spikes_df = _prepare_spikes_df_from_filter_epochs(long_session.spikes_df, filter_epochs=filter_epochs, included_neuron_ids=curr_any_context_neurons, epoch_id_key_name='replay_epoch_id', debug_print=False) # replay_epoch_id
filter_epoch_spikes_df

In [ ]:
from pyphocorehelpers.indexing_helpers import partition

unique_replay_epoch_id_values, epoch_split_spikes_df_list = partition(filter_epoch_spikes_df, 'replay_epoch_id')
# filter_epoch_spikes_df.groupby('replay_epoch_id')
unique_replay_epoch_id_values # (198,)

In [ ]:
len(epoch_split_spikes_df_list)

In [ ]:
len(epoch_spikes_unique_aclus)
len(epoch_contains_any_exclusive_aclus)
len(filter_epochs)

In [ ]:
# find epochs containing at least one in `exclusive_aclus`:
epoch_spikes_unique_aclus = [epoch_split_spikes_df.aclu.unique() for epoch_split_spikes_df in epoch_split_spikes_df_list]
# epoch_spikes_unique_aclus # finds lists of all the active aclus in the given epoch
epoch_contains_any_exclusive_aclus = [np.isin(epoch_aclus, exclusive_aclus).any() for epoch_aclus in epoch_spikes_unique_aclus]
# epoch_contains_any_exclusive_aclus # returns True if the epoch contains one ore more spikes from the search list `exclusive_aclus`
filter_epochs['contains_one_exclusive_aclu'] = epoch_contains_any_exclusive_aclus
# epoch_contains_any_exclusive_aclus

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_multiple_raster_plot

app, win, plots, plots_data = plot_multiple_raster_plot(filter_epochs_df, filter_epoch_spikes_df, included_neuron_ids=shared_aclus, epoch_id_key_name='replay_epoch_id', scatter_app_name="Pho Stacked Replays")